In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Detectron2 installation.

In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git
!python -m pip install -e detectron2

Version Controls

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()


import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow


from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

Register

In [ ]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("my_dataset_train", {}, "path_train_annotations.json", "path_your_train_dataset")
register_coco_instances("my_dataset_val", {}, "path_val_annotations.json", "path_your_val_dataset")

In [ ]:
train_metadata = MetadataCatalog.get("my_dataset_train")
train_dataset_dicts = DatasetCatalog.get("my_dataset_train")

In [ ]:
val_metadata = MetadataCatalog.get("my_dataset_val")
val_dataset_dicts = DatasetCatalog.get("my_dataset_val")

Visualizing Some Random Examples.

In [ ]:
from matplotlib import pyplot as plt

for d in random.sample(train_dataset_dicts, 2):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    plt.imshow(vis.get_image()[:, :, ::-1])
    plt.axis(False)
    plt.show()

In [ ]:
from detectron2.engine import DefaultTrainer

# HYPERPARAMETERS
MODEL = "path/to/mask2former/pkl"
CONFIG = "path/to/mask2former/yaml"

cfg = get_cfg()
cfg.OUTPUT_DIR = "output_path"
cfg.merge_from_file = (CONFIG)
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val")
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = MODEL
cfg.SOLVER.IMS_PER_BATCH = 4  # Batch size
cfg.SOLVER.BASE_LR = 0.0001  # Learning_rate
cfg.INPUT.MASK_FORMAT='polygon'
cfg.SOLVER.MAX_ITER = 5    # Iteration
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False) # If there is a pre-trained model, you can upload it.

In [ ]:
trainer.train() # Start train


Save Config File

In [ ]:
import yaml
config_yaml_path = "path_output/output.yaml" # Should be output.yaml at the end
with open(config_yaml_path, 'w') as file:
    yaml.dump(cfg, file)

### Inference & Evaluation

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Threshold
predictor = DefaultPredictor(cfg)

Segment Random Images in Validation Dataset

In [ ]:
from detectron2.utils.visualizer import ColorMode
                       #val
for d in random.sample(val_dataset_dicts, 10):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=val_metadata,
                   scale=0.5,
                   instance_mode=ColorMode.IMAGE_BW
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

Metric Scores

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

Upload New Image

In [ ]:
new_im = cv2.imread("new_image.jpg")
outputs  = predictor(new_im)

v = Visualizer(new_im[:, :, ::-1], metadata=train_metadata)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

cv2_imshow(out.get_image()[:, :, ::-1])


Segmenting and Saving Multiple Photos in a Folder

In [ ]:
input_images_directory = "test_path"

output_directory = "test/test_results"


for image_filename in os.listdir(input_images_directory):
    image_path = os.path.join(input_images_directory, image_filename)
    new_im = cv2.imread(image_path)

    outputs = predictor(new_im)

    v = Visualizer(new_im[:, :, ::-1], metadata=train_metadata)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    result_filename = os.path.splitext(image_filename)[0] + "_result.png"
    output_path = os.path.join(output_directory, result_filename)

    cv2.imwrite(output_path, out.get_image()[:, :, ::-1])

print("All photos saved successfully!")

In [ ]:
import csv
from skimage.measure import regionprops, label


# Image folder path
input_images_directory = "test"

# Where to save the CSV file
output_csv_path = "test_results/output_objects.csv"  # Replace this with the path to your desired output CSV file


with open(output_csv_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)


    csvwriter.writerow(["File Name", "Class Name", "Object Number", "Area", "Centroid", "BoundingBox"])  # Add more columns as needed for other properties

    for image_filename in os.listdir(input_images_directory):
        image_path = os.path.join(input_images_directory, image_filename)
        new_im = cv2.imread(image_path)

        outputs = predictor(new_im)

        mask = outputs["instances"].pred_masks.to("cpu").numpy().astype(bool)


        class_labels = outputs["instances"].pred_classes.to("cpu").numpy()

       # # Debugging: print class_labels and metadata.thing_classes
       # #print("Class Labels:", class_labels)
       # #print("Thing Classes:", train_metadata.thing_classes)

        labeled_mask = label(mask)
        props = regionprops(labeled_mask)

        for i, prop in enumerate(props):
            object_number = i + 1  # object number = 1 (i = 0 / i+1 = 1)
            area = prop.area
            centroid = prop.centroid
            bounding_box = prop.bbox

            if i < len(class_labels):
                class_label = class_labels[i]
                class_name = train_metadata.thing_classes[class_label]
            else:
                class_name = 'Unknown'

            csvwriter.writerow([image_filename, class_name, object_number, area, centroid, bounding_box])  # Add more columns as needed for other properties

print("Object Information Saved to CSV.")


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

csv_file_path = "test_results/output_objects.csv"  # Update with your CSV file path

df = pd.read_csv(csv_file_path)

class_names = train_metadata.thing_classes


avg_objects_per_class = df.groupby(["File Name", "Class Name"])["Object Number"].count().reset_index()
avg_objects_per_class = avg_objects_per_class.groupby("Class Name")["Object Number"].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x="Class Name", y="Object Number", data=avg_objects_per_class, ci=None, order=class_names)
plt.xticks(rotation=45)
plt.xlabel("Class Name")
plt.ylabel("Average Number of Objects per Image")
plt.title("Average Number of Objects per Image for Each Class")
plt.tight_layout()
plt.show()


avg_area_per_class = df.groupby("Class Name")["Area"].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x="Class Name", y="Area", data=avg_area_per_class, ci=None, order=class_names)
plt.xticks(rotation=45)
plt.xlabel("Class Name")
plt.ylabel("Average Area of Objects")
plt.title("Average Area of Objects for Each Class")
plt.tight_layout()
plt.show()

Saving Binary Masks

In [ ]:
import os
import cv2
import numpy as np
import torch
from detectron2.utils.visualizer import Visualizer

input_images_directory = "test"

output_directory = "test_result_instance"  # Replace this with the path to your desired output directory

for image_filename in os.listdir(input_images_directory):
    image_path = os.path.join(input_images_directory, image_filename)
    new_im = cv2.imread(image_path)

    outputs = predictor(new_im)

    class_masks = {class_name: torch.zeros_like(outputs["instances"].pred_masks[0], dtype=torch.uint8, device=torch.device("cuda:0"))
                   for class_name in train_metadata.thing_classes}

    for i, pred_class in enumerate(outputs["instances"].pred_classes):
        class_name = train_metadata.thing_classes[pred_class]
        class_masks[class_name] = torch.where(outputs["instances"].pred_masks[i].to(device=torch.device("cuda:0")),
                                              i + 1,
                                              class_masks[class_name])

    for class_name, class_mask in class_masks.items():

        class_mask_np = class_mask.cpu().numpy()

        class_filename = os.path.splitext(image_filename)[0] + f"_{class_name}_result.png"
        class_output_path = os.path.join(output_directory, class_filename)

        cv2.imwrite(class_output_path, class_mask_np.astype(np.uint8))

print("Segmentation of All Images Completed!")